# 머신러닝 모델 구성

- 케라스와 토치 모델 만드는 방법을 배운다

# import

In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 7.2 MB/s 
     |████████████████████████████████| 36.8 MB 35 kB/s 


In [ ]:
import deepchem as dc
import tensorflow as tf
import torch

# 모델 구성 방법 (회귀)
- DeepChem's `Dataset`이 다른 프레임워크와 연결하는 방법을 제공한다
 - `make_tf_dataset()` 은 데이터를 가져오기 위한 `tensorflow.data.Dataset`를 얻는다
 - `make_pytorch_dataset()`은 `torch.utils.data.IterableDataset`을 얻는데 사용한다
 - 이러한 함수들을 사용하면 DeepChem의 datasets, loaders, featurizers, transformers, splitters 등을 사용할 수 있다



## KerasModel

- dc.models 모듈이 제공하는 KerasModel을 사용하면 케라스 모델을 만들 수 있다.
- 아래는 히든 계층이 하나 있는 MLP 모델 정의이다
- 손실함수로 L2Loss를 지정했다

In [ ]:
keras_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])
model = dc.models.KerasModel(keras_model, dc.models.losses.L2Loss())

# 모델 학습과 성능 평가

In [ ]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='ECFP', splitter='random')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=50)
metric = dc.metrics.Metric(dc.metrics.r2_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'r2_score': 0.9776049089758412}
test set score: {'r2_score': 0.7377961323439772}


## TorchModel

- 앞과 같은 구조를 갖는 모델을 torch로 만들어보겠다
- `TorchModel` 은 `KerasModel`와 같은 기능을 하며 `torch.nn.Module`을 내부에 사용한다

In [ ]:
pytorch_model = torch.nn.Sequential(
    torch.nn.Linear(1024, 1000),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1000, 1)
)
model = dc.models.TorchModel(pytorch_model, dc.models.losses.L2Loss())

model.fit(train_dataset, nb_epoch=50)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'r2_score': 0.977441602580224}
test set score: {'r2_score': 0.7288244030115891}


# 모델 클래스 정의 방법 (분류)

- 모델을 클래스로 작성하는 방법을 소개
- 선형 결과 값에 시그모이드를 수행하는 이진 분류 모델을 만드는 클래스를 정의한다


## 케라스 모델을 사용하는 경우

In [ ]:
class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(1000, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs, training=False):
        y = self.dense1(inputs)
        if training:
            y = tf.nn.dropout(y, 0.5)
        logits = self.dense2(y)
        output = tf.nn.sigmoid(logits)
        return output, logits

keras_model = MyModel()
output_types = ['prediction', 'loss']
model = dc.models.KerasModel(keras_model, 
      dc.models.losses.SigmoidCrossEntropy(), output_types=output_types)

- 테스트를 위해서 BACE dataset에 적용해보았다
 - 분자가 BACE-1 효소를 저해하는지를 예측하는 이진 분류 데이터

In [ ]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(feturizer='ECFP', splitter='scaffold')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=50)
metric1 = dc.metrics.Metric(dc.metrics.roc_auc_score)
metric2 = dc.metrics.Metric(dc.metrics.accuracy_score)
print('training set score:', model.evaluate(train_dataset, [metric1, metric2]))
print('test set score:', model.evaluate(test_dataset, [metric1, metric2]))

training set score: {'roc_auc_score': 0.9995641545016414, 'accuracy_score': 0.9867768595041322}
test set score: {'roc_auc_score': 0.7814764492753623, 'accuracy_score': 0.6842105263157895}


## 토치 모델을 사용하는 경우

In [ ]:
class MyTorchModel(torch.nn.Module):
    
    def __init__(self):
        super(MyTorchModel, self).__init__()
        self.dense1 = torch.nn.Linear(1024, 1000)
        self.dense2 = torch.nn.Linear(1000, 1)

    def forward(self, inputs):
        y = torch.nn.functional.relu( self.dense1(inputs) )
        y = torch.nn.functional.dropout(y, p=0.5, training=self.training)
        logits = self.dense2(y)
        output = torch.sigmoid(logits)
        return output, logits

torch_model = MyTorchModel()
output_types = ['prediction', 'loss']
model = dc.models.TorchModel(torch_model, dc.models.losses.SigmoidCrossEntropy(), output_types=output_types)

In [ ]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(feturizer='ECFP', splitter='scaffold')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=50)
metric1 = dc.metrics.Metric(dc.metrics.roc_auc_score)
metric2 = dc.metrics.Metric(dc.metrics.accuracy_score)
print('training set score:', model.evaluate(train_dataset, [metric1, metric2]))
print('test set score:', model.evaluate(test_dataset, [metric1, metric2]))

training set score: {'roc_auc_score': 0.9995948871970385, 'accuracy_score': 0.9867768595041322}
test set score: {'roc_auc_score': 0.7715579710144929, 'accuracy_score': 0.6776315789473685}


# Splitters

- 훈련, 검증, 테스트 데이터 나누기

## 종류
- RandomSplitter
 - 랜덤하게 데이터를 나누는 방법

- RandomStratifiedSplitter
 - 랜덤하게 나누되 레이블 분포의 비율에 맞게 나누는 방법

- ScaffoldSplitter
 - 주어진 데이터중에 구조적으로 매우 비슷한 샘플들이 많은 경우 훈련 데이터와 검증데이터에 유사한  데이터가 나뉘어 들어가면 성능이 좋아지는 문제가 있다.
 - 이러한 문제를 피하기 위해서 분자 구조가 유사한 스카폴드들을 묶어서 훈련과 검증 데이터에 섞이지 않도록 하는 방법
 
- ButinaSplitter
 - 유사 샘플이 많은 문제를 피하기 위한 방법으로 비슷한 Fingerprint를 가진 분자들을 같은 클러스터링으로 묶는 방법(계산 시간이 필요하다)

- SpecifiedSplitter
 - 최적의 분할 구성을 미리 알고 지정할 때, 또는 시계열 데이터 분할


In [ ]:
splitters = ['random', 'scaffold', 'butina']
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
for splitter in splitters:
    tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP', splitter=splitter)
    train_dataset, valid_dataset, test_dataset = datasets
    model = dc.models.MultitaskClassifier(n_tasks=len(tasks), n_features=1024, layer_sizes=[1000])
    model.fit(train_dataset, nb_epoch=20)
    print('splitter:', splitter)
    print('training set score:', model.evaluate(train_dataset, [metric], transformers))
    print('test set score:', model.evaluate(test_dataset, [metric], transformers))
    print()

splitter: random
training set score: {'roc_auc_score': 0.9732972349448863}
test set score: {'roc_auc_score': 0.7735860163829839}

splitter: scaffold
training set score: {'roc_auc_score': 0.9756677196925813}
test set score: {'roc_auc_score': 0.6758625674350305}

splitter: butina
training set score: {'roc_auc_score': 0.9757419325575071}
test set score: {'roc_auc_score': 0.6010516860209848}



- (주의) 위결과만 모고 랜덤 분할이 가장 좋다고 단정할 수 없다

# 멀티태스크 모델

- 딥러닝 모델은 멀티태스크 (다중 레이블링) 구현이 용이하다
-  데이터 수가 적거나, 비대칭성일 때 멀티태스킹 작업이 성능을 개선시킬 수 있다

## MUV 데이터 이용 예
- MUV dataset에는 17개의 타겟에 대해서 소수의 액티브 샘플만 보유하고 있다
- 총 93,087 개의 화합물이 있는데 태스크별로 30개 이하의 액티브 샘플만 존재한다
- 멀티태스크 모델을 사용하여 이러한 문제를 일부 개선할 수 있다. 한가지 태스크 예측에 도움이 되는 특성은 다른 태스크에도 도움이 될 것임

In [ ]:
tasks, datasets, transformers = dc.molnet.load_muv(splitter='stratified')
train_dataset, valid_dataset, test_dataset = datasets

In [ ]:
n_tasks = len(tasks)
n_features = train_dataset.get_data_shape()[0]
model = dc.models.MultitaskClassifier(n_tasks, n_features)
model.fit(train_dataset)

0.45682652791341144

In [ ]:
y_true = test_dataset.y
y_pred = model.predict(test_dataset)
metric = dc.metrics.roc_auc_score
for i in range(n_tasks):
    score = metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred[:,i])
    print(tasks[i], score)

MUV-466 0.722478953967401
MUV-548 0.8680368977252373
MUV-600 0.6440533763209744
MUV-644 0.9724879097259538
MUV-652 0.8235088662009673
MUV-689 0.761114096363962
MUV-692 0.3533046749059645
MUV-712 0.7318287658964714
MUV-713 0.48229446534121445
MUV-733 0.5551226938921726
MUV-737 0.3838169442951818
MUV-810 0.41215296435608095
MUV-832 0.9171234103528569
MUV-846 0.9993283181085438
MUV-852 0.869801182160129
MUV-858 0.5088930682428803
MUV-859 0.7095556630131099


# 모델 최적화

- `dc.hyper` 패키지가 제공하는 하이퍼파라미터 최적화기 `GridHyperparamOpt`에 하이퍼파라미터 리스트를 입력하면 모든 경우의 수를 시도해본다
- HIV dataset 사용
 - 40,000 이상 분자 데이터로 inhibit HIV replication 예측하는 모델

In [ ]:
tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', splitter='scaffold')
train_dataset, valid_dataset, test_dataset = datasets

In [ ]:
tasks

['HIV_active']

In [ ]:
params_dict = {
    'n_tasks': [len(tasks)],
    'n_features': [1024],
    'layer_sizes': [[500], [1000], [1000, 1000]],
    'dropouts': [0.2, 0.5],
    'learning_rate': [0.001, 0.0001]
}
optimizer = dc.hyper.GridHyperparamOpt(dc.models.MultitaskClassifier)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
        params_dict, train_dataset, valid_dataset, metric, transformers)

- `hyperparam_search()`는 다음의 세가지를 리턴한다
 - 찾아낸 best model
 - 해당 모델의 하이퍼파라이터
 - 모든 모델의 검증 점수 전체 리스트

In [ ]:
best_model

In [ ]:
best_hyperparams

{'n_tasks': 1,
 'n_features': 1024,
 'layer_sizes': [500],
 'dropouts': 0.2,
 'learning_rate': 0.001}

In [ ]:
all_results

{'_dropouts_0.200000_layer_sizes[500]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7899803118263766,
 '_dropouts_0.200000_layer_sizes[500]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7745949074074074,
 '_dropouts_0.500000_layer_sizes[500]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7707001396237507,
 '_dropouts_0.500000_layer_sizes[500]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7573103444052518,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7848339824612973,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7682046712718009,
 '_dropouts_0.500000_layer_sizes[1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7798828507740545,
 '_dropouts_0.500000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7624421296296298,
 '_dropouts_0.200000_layer_sizes[1000, 1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7508236576523613,
 '_dropo

# Early Stopping

- `ValidationCallback`를 사용하여 성능을 기록해준다(예: 1000 step 마다). `save_dir` 인자를 사용하면 최적의 하이퍼파라미터를 디스크에 저장한다

In [ ]:
len(train_dataset)

32901

In [ ]:
model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                                      n_features=1024,
                                      layer_sizes=[500],
                                      dropouts=0.2,
                                      learning_rate=0.001)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
callback = dc.models.ValidationCallback(valid_dataset, 1000, metrics=metric)
model.fit(train_dataset, nb_epoch=50, callbacks=callback)

Step 1000 validation: roc_auc_score=0.718167
Step 2000 validation: roc_auc_score=0.774653
Step 3000 validation: roc_auc_score=0.760219
Step 4000 validation: roc_auc_score=0.766241
Step 5000 validation: roc_auc_score=0.769059
Step 6000 validation: roc_auc_score=0.770381
Step 7000 validation: roc_auc_score=0.762457
Step 8000 validation: roc_auc_score=0.772279
Step 9000 validation: roc_auc_score=0.772578
Step 10000 validation: roc_auc_score=0.746568
Step 11000 validation: roc_auc_score=0.754964
Step 12000 validation: roc_auc_score=0.7482
Step 13000 validation: roc_auc_score=0.728815
Step 14000 validation: roc_auc_score=0.733774
Step 15000 validation: roc_auc_score=0.72005
Step 16000 validation: roc_auc_score=0.718444


0.028371157646179198

# 학습률 스케쥴

- 학습이 진행되면서 학습률을 줄여나간다
- `ExponentialDecay` 객체를 사용하면 된다 
 - 예를 들면 0.0002에서 출발하여 1000 스텝마다 0.9배로 줄여나간다

In [ ]:
learning_rate = dc.models.optimizers.ExponentialDecay(0.0002, 0.9, 1000)
model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                                      n_features=1024,
                                      layer_sizes=[1000],
                                      dropouts=0.2,
                                      learning_rate=learning_rate)
model.fit(train_dataset, nb_epoch=50, callbacks=callback)

In [ ]:
model.evaluate(valid_dataset, [metric])

{'roc_auc_score': 0.680811226239467}